### Long memory

Generate datasets

In [3]:
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_json("msc_self_instruct.jsonl", lines=True)

answer_df = pd.DataFrame()
answer_df['dialog'] = None
answer_df['question'] = None
answer_df['gold_answer'] = None
answer_df['long_mem_result'] = None
answer_df['long_mem_answer'] = None
answer_df['long_mem_f1'] = None
answer_df['long_mem_rc'] = None
answer_df['long_mem_pre'] = None
answer_df['long_mem_recall_result'] = None
answer_df['long_mem_recall_answer'] = None
answer_df['long_mem_recall_f1'] = None
answer_df['long_mem_recall_rc'] = None
answer_df['long_mem_recall_pre'] = None

for i in range(500):
    # Add dialog
    current_time = datetime.strptime("2024/11/1 12:00", "%Y/%m/%d %H:%M")
    dialog_data = []
    for dialog_session in df['previous_dialogs'][i]:
        session = []
        for count in range(int(len(dialog_session['dialog'])/2)):
            chat_log = {"text":f"user:{dialog_session['dialog'][2*count]['text']}, assistant:{dialog_session['dialog'][2*count+1]['text']}", "time":current_time.strftime("%Y-%m-%dT%H:%M:%SZ")}
            session.append(chat_log)
            current_time = current_time + timedelta(minutes=1)
        dialog_data.append(session)
    answer_df.loc[i, 'dialog'] = str(dialog_data)
    
    # Add question & answer
    answer_df.loc[i, 'question'] = f"{df['self_instruct'][i]['B']}"
    answer_df.loc[i, 'gold_answer'] = f"{df['self_instruct'][i]['A']}"
answer_df.to_json("MSC_datasets.json", orient="records", lines=True)

In [2]:
import pandas as pd

answer_df = pd.read_json("MSC_datasets.json", lines=True)
# answer_df = pd.read_json("MSC_eval.json", lines=True)

In [5]:
from long_memory.component import WeaviateLongMemory
long_mem = WeaviateLongMemory()

Detect existed deafult user group memory space, loading...
Detect existed deafult user child memory space, loading...


In [3]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def llm_create(prompt):
        messages = [{"role": "user", "content": prompt}]
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0
        )
        return completion.choices[0].message.content

# classify_instruct="""The summary should focus on keywords such as personal preferences, experiences, activities, and locations."""

# recall_instruct="""You will get a question about user. find relative memory as more as possible, don't give up easily"""

answer_prompt = """Base on the following document and answer the question.
Assistant will ask you a question, give an detail answer if you are User

Question:{question}

Document:{docs}
"""

In [7]:
for i in range(100, 500):
    print(f'==={i}===')
    success = False
    while not success:
        try:
            # generate result
            long_mem.del_memory()
            dialogs = []
            for dialog_session in eval(answer_df.loc[i, 'dialog']):
                dialogs.extend(dialog_session)
            long_mem.add_chat_logs(dialogs) # other_instruct=classify_instruct
            question = f"assistant:{answer_df.loc[i, 'question']}"
            answer_df.loc[i, 'long_mem_result'] = str(long_mem.get_memory(question, recall=False))
            answer_df.loc[i, 'long_mem_recall_result'] = str(long_mem.get_memory(question, recall=True)) # other_instruct=recall_instruct
            
            # response
            p = answer_prompt.format(question=answer_df.loc[i, 'question'], docs=answer_df.loc[i, 'long_mem_result'])
            long_mem_answer = llm_create(p)
            answer_df.loc[i, 'long_mem_answer'] = long_mem_answer
            
            p = answer_prompt.format(question=answer_df.loc[i, 'question'], docs=answer_df.loc[i, 'long_mem_recall_result'])
            long_mem_recall_answer = llm_create(p)
            answer_df.loc[i, 'long_mem_recall_answer'] = long_mem_recall_answer
            
            success = True
        except Exception as e:
            print(f'----error:{e}----')
    if (i+1)%20==0:
        answer_df.to_json("MSC_eval.json", orient="records", lines=True)
answer_df.to_json("MSC_eval.json", orient="records", lines=True)

===100===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save chat logs to long memory done.
===101===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save chat logs to long memory done.
===102===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Chat logs not correct, missing id:{3}, unknown id:set(), retry..
Save chat logs to long memory done.
===103===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Chat logs not correct, missing id:{1}, unknown id:set(), retry..
Chat logs not correct, missing id:{1}, unknown id:set(), retry..
Save chat logs to long memory done.
===104===
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
Save chat logs to long memory done.
===105===
Detect empty group memory, create memory space...
Detect empty ch

### Short memory

In [10]:
import pandas as pd
from datetime import datetime, timedelta
df = pd.read_json("msc_self_instruct.jsonl", lines=True)

s_answer_df = pd.DataFrame()
s_answer_df['dialog'] = None
s_answer_df['question'] = None
s_answer_df['gold_answer'] = None
s_answer_df['short_mem_result'] = None
s_answer_df['short_mem_answer'] = None
s_answer_df['short_mem_f1'] = None
s_answer_df['short_mem_rc'] = None
s_answer_df['short_mem_pre'] = None

for i in range(500):
    # Add dialog
    current_time = datetime.strptime("2024/11/1 12:00", "%Y/%m/%d %H:%M")
    dialog_data = []
    for dialog_session in df['previous_dialogs'][i]:
        session = []
        for count in range(int(len(dialog_session['dialog'])/2)):
            chat_log = {"assistant":f"{dialog_session['dialog'][2*count]['text']}", 
                        "user":f"{dialog_session['dialog'][2*count+1]['text']}", 
                        "time":current_time.strftime("%Y-%m-%dT%H:%M:%SZ")}
            session.append(chat_log)
            current_time = current_time + timedelta(minutes=1)
        dialog_data.append(session)
    s_answer_df.loc[i, 'dialog'] = str(dialog_data)
    
    # Add question & answer
    s_answer_df.loc[i, 'question'] = f"user:{df['self_instruct'][i]['B']}"
    s_answer_df.loc[i, 'gold_answer'] = f"{df['self_instruct'][i]['A']}"
s_answer_df.to_json("s_MSC_datasets.json", orient="records", lines=True)

In [15]:
import pandas as pd

s_answer_df = pd.read_json("s_MSC_datasets.json", lines=True)
# s_answer_df = pd.read_json("s_MSC_eval.json", lines=True)

In [2]:
from short_memory.component import WeaviateShortMemory
short_mem = WeaviateShortMemory()

Detect existed deafult user short memory space, loading...


In [13]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def llm_create(prompt):
        messages = [{"role": "user", "content": prompt}]
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        return completion.choices[0].message.content

answer_prompt = """Base on the following document and answer the question.
Try use origin text in the evidence field to repeat a brief answer.

Question:{question}

Document:{docs}
"""

In [ ]:
error_list = []
for i in range(500):
    print(f'==={i}===')
    try:
        # generate result
        short_mem.del_memory()
        dialogs = []
        for dialog_session in eval(s_answer_df.loc[i, 'dialog']):
            dialogs.extend(dialog_session)
        short_mem.add_chatlogs(dialogs)
        question = f"{s_answer_df.loc[i, 'question']}"
        s_answer_df.loc[i, 'short_mem_result'] = str(short_mem.get_memory(question))
        
        # response
        p = answer_prompt.format(question=s_answer_df.loc[i, 'question'], docs=s_answer_df.loc[i, 'short_mem_result'])
        short_mem_answer = llm_create(p)
        s_answer_df.loc[i, 'short_mem_answer'] = short_mem_answer
    except Exception as e:
        error_list.append(i)
        print(f'----error:{e}----')
    if (i+1)%20==0:
        s_answer_df.to_json("s_MSC_eval.json", orient="records", lines=True)
s_answer_df.to_json("s_MSC_eval.json", orient="records", lines=True)